In [319]:
import torch
import torch.autograd as autograd
import torch.nn as nn
from torch import Tensor
import torch.optim as optim


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from sklearn.model_selection import train_test_split

import numpy as np
import time
import scipy.io

torch.set_default_dtype(torch.float)

# Random number generators in other libraries
np.random.seed(28)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 

cpu


In [320]:
# Lucas/this_script.py
import sys
import os
current_dir = os.getcwd()
print(current_dir)
# Add the 'functions' folder to the Python path
functions_dir = os.path.join(current_dir,'..', 'functions')
sys.path.append(functions_dir)
functions_dir = os.path.join(current_dir,'..', 'Mads')
sys.path.append(functions_dir)

# Now you can import the function
from Load_data import custom_csv_parser2, custom_csv_parser
from Load_data import data_split

pde_keys = ['D1', 'D2', 'I_sc','I_p', 'I_eff', 'G', 'G_sc']
patient_keys = ['tau1', 'tau2', 'Ci', 'p2', 'GEZI', 'EGP0', 'Vg', 'taum', 'tausc']
patient_keys_si = ['tau1', 'tau2', 'Ci', 'p2', 'Si', 'GEZI', 'EGP0', 'Vg', 'taum', 'tausc']

def dict_to_vec(X, t):
    """Converts a dictionary of time series data to a vector"""
    t = t.cpu().numpy().astype(int) if t.numel() > 1 else [int(t.item())]  # Ensure t is a list of integers
    X_vec = torch.zeros((len(t), len(pde_keys)), device=device)  # Initialize X_vec with the correct shape
    
    for i, ti in enumerate(t):
        for j, key in enumerate(pde_keys):
            value = torch.tensor(X[key][ti], device=device)  # Convert to torch.Tensor
            X_vec[i, j] = value
    
    return X_vec.to(device)
scale_vec = torch.tensor([1.56, 1.54, 1.24, 1.24, 0.01, 117.56, 117.48], device=device, dtype=torch.float32)
# Use the function
data = custom_csv_parser2('../Patient2.csv')
parms = custom_csv_parser('../Patient.csv')
Si_true = parms['Si']

X_train, X_test, ts_train, ts_test, ts = data_split(data)
G_true = X_train["G"]


c:\DeepLearning-PINNs\Lucas
Training data loaded successfully!


# Hyperparamters

In [321]:
Si0 = 0.01

p = 0.4
lr = 0.001

In [322]:
class DNN(nn.Module):
    '''Network'''
    def __init__(self, layers):
        super(DNN, self).__init__()

        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1] ) for i in range(len(layers)-1)])

        self.activation = nn.Tanh()

        self.dropout = nn.Dropout(p=p)

        'Initialization'
        for i in range(len(layers) - 1):
            nn.init.xavier_normal_(self.linears[i].weight, gain=1.0)
            nn.init.zeros_(self.linears[i].bias)

    def forward(self, x):
        for i in range(0, len(self.linears) - 1):
            x = self.linears[i](x)
            x = self.activation(x)
            x = self.dropout(x)
        x = self.linears[-1](x)
        return x

In [323]:
def pdes(u, nn_dt, Si):
    pdes = torch.zeros_like(u, device=device)
    D1 = u[:, 0]  
    D2 = u[:, 1]  
    I_sc = u[:, 2]  
    I_p = u[:, 3]  
    I_eff = u[:, 4]  
    G = u[:, 5]  
    G_sc = u[:, 6]  
    
    # Convert parameters to tensors
    taum = torch.tensor(parms['taum'], device=device)
    tau1 = torch.tensor(parms['tau1'], device=device)
    tau2 = torch.tensor(parms['tau2'], device=device)
    p2 = torch.tensor(parms['p2'], device=device)
    Ci = torch.tensor(parms['Ci'], device=device)
    GEZI = torch.tensor(parms['GEZI'], device=device)
    EGP0 = torch.tensor(parms['EGP0'], device=device)
    Vg = torch.tensor(parms['Vg'], device=device)
    tausc = torch.tensor(parms['tausc'], device=device)

    ds = torch.zeros_like(u[:,0], device=device)
    ds[0] = data['Meal'][0]
    us = torch.ones_like(u[:,0], device=device) * 25.04
    us[0] = data['Insulin'][0]
    
    # Example assignment for all rows of pdes
    pdes[:, 0] = nn_dt[:, 0] + D1 / taum - ds
    pdes[:, 1] = nn_dt[:, 1] - (D1 - D2) / taum
    pdes[:, 2] = nn_dt[:, 2] + I_sc / tau1 - us / (tau1 * Ci)
    pdes[:, 3] = nn_dt[:, 3] - (I_sc - I_p) / tau2
    pdes[:, 4] = nn_dt[:, 4] + p2 * I_eff - p2 * Si * I_p
    pdes[:, 5] = nn_dt[:, 5] + (GEZI + I_eff) * G - EGP0 - 1000 * D2 / (Vg * taum)
    pdes[:, 6] = nn_dt[:, 6] - (G - G_sc) / tausc

    return pdes / scale_vec

class PINN():
    def __init__(self, layers):
        self.iter = 0

        self.Si = torch.tensor([Si0], requires_grad=True, device=device)
        self.Si = nn.Parameter(self.Si)
        self.nn = DNN(layers).to(device)
        self.nn.register_parameter('Si', self.Si)

    def data_loss(self, ts, data):
        u = self.nn(ts)  # Outputs ts x 7 array
        loss = 0
        for i,key in enumerate(pde_keys):
            loss += torch.mean((u[i] - data[key][int(ts[i])])**2)
        return loss

    def pde_loss(self, ts):
        
        ts = ts.clone().detach()
        ts = ts.requires_grad_(True)
        u = self.nn(ts)        
        nn_dt = torch.zeros_like(u, device=device)

        for i in range(u.shape[1]):
            nn_dt[:, i] = autograd.grad(u[:, i], ts, grad_outputs=torch.ones_like(u[:, i]), create_graph=True)[0].squeeze()

        pd = pdes(u, nn_dt, self.Si)
        loss = nn.MSELoss()(pd, torch.zeros_like(pd))
        return loss
    
    def loss(self, ts, data):
        return self.data_loss(ts, data) + self.pde_loss(ts)
    
    def test(self, t):
        #t = t.unsqueeze(0)
        X_pred = self.nn.forward(t)
        X_true = dict_to_vec(data, t)
        print(X_pred.shape)
        print(X_true.shape)
        error_vec = torch.linalg.norm((X_true-X_pred),2)/torch.linalg.norm(X_true,2)
        X_pred = X_pred.cpu().detach().numpy()
        return X_pred, error_vec


In [324]:
NN = PINN([1, 20, 40, 20, 7])
params = list(NN.nn.parameters())
optimizer = optim.Adam(NN.nn.parameters(), lr=0.001)

'''Test Network works'''
pred, err = NN.test(torch.tensor([1], device=device, dtype=torch.float32))
print(f"""Prediction: {pred} \n Error: {err}""")
print("Current guess for Si: ", NN.Si.item())


torch.Size([7])
torch.Size([1, 7])
Prediction: [-0.57569087 -0.09128979 -0.37735116  0.95070064 -0.39105302  1.4893415
 -0.25105685] 
 Error: 0.9945842623710632
Current guess for Si:  0.009999999776482582


In [325]:
start_time = time.time()
ts_train = torch.tensor(ts_train, device=device, dtype=torch.float32).reshape(-1,1)
ts_test = torch.tensor(ts_test, device=device, dtype=torch.float32)
epochs = 3000

for epoch in range(epochs):
    optimizer.zero_grad()
    loss = NN.loss(ts_train, parms)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}, Si: {NN.Si.item()}")

print("True Si: ", Si_true)

KeyError: 'Meal'

1.564036816889645
1.5472147500856661
1.2474305980382763
1.247415539752733
0.010104967942474905
117.560994471807
117.48980875234228
